In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
#from lifelines import CoxPHFitter
#from lifelines.datasets import load_rossi
#import statsmodels.api as sm
#from statsmodels.formula.api import ols
#import statsmodels.stats.multicomp

%matplotlib inline

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
ehr = pd.read_csv('/kaggle/input/full data - dm_longitudinal_062118.csv', delimiter=',', dtype={'study_id': 'category', 'smoke': 'category', 'PAT_STATUS_C': 'category', 'CVD': 'category'})
ehr = ehr.drop(ehr.columns[0], axis=1)

In [ ]:
ehr['Lab'] = ehr.astype({"ENCOUNTER_DT_RAN": str}).apply(lambda x: [x['LAB_TEST'], x['LAB_RESULT']], axis = 1)

In [ ]:

#print (pd.DataFrame.from_records(a.values.tolist()).stack().value_counts())
treatment_mapping = ehr[['THERA_CLASS','PHARM_CLASS']].dropna().drop_duplicates()
#ehr[['PHARM_CLASS']].dropna().drop_duplicates().count()
ehr.columns


In [ ]:
# diabetes_therapy_list = ehr.loc[ehr['RX_CATEGORY'] == 'Diabetes RX',]['PHARM_CLASS'].unique()

# # 'PHARM_CLASS', 'RX_CATEGORY', 'THERA_CLASS'
# therapy = ehr.groupby(['ENCOUNTER_DT_RAN', 'study_id', 'SEX', 'AGE_END2017', 'RACE', 'ETHNICITY', 'smoke', 'BP_SYSTOLIC', 'BP_DIASTOLIC' ,'BMI', 'WEIGHT'])['PHARM_CLASS'].apply(list)
# therapies = []
# for list_therapy in therapy.values.tolist():
#     new_list = set()
#     for t in list_therapy:
#         if t in diabetes_therapy_list:
#             new_list.add(t)
#     therapies.append(list(new_list))
# group by visit
lab = ehr.groupby(['ENCOUNTER_DT_RAN', 'study_id', 'SEX', 'AGE_END2017', 'RACE', 'ETHNICITY', 'smoke', 'BP_SYSTOLIC', 'BP_DIASTOLIC' ,'BMI', 'WEIGHT'])['Lab'].apply(list)
lab_test = pd.DataFrame({'ENCOUNTER_DT_RAN': lab.index.get_level_values(0).tolist(),
                         'study_id': lab.index.get_level_values(1).tolist(),
                         'sex': lab.index.get_level_values(2).tolist(), 
                         'age': lab.index.get_level_values(3).tolist(), 
                         'race': lab.index.get_level_values(4).tolist(), 
                         'ethnicity': lab.index.get_level_values(5).tolist(),
                         'smoke': lab.index.get_level_values(6).tolist(),
                         'BP_SYSTOLIC': lab.index.get_level_values(7).tolist(),
                         'BP_DIASTOLIC': lab.index.get_level_values(8).tolist(),
                         'BMI': lab.index.get_level_values(9).tolist(), 
                         'WEIGHT': lab.index.get_level_values(10).tolist(),
                         #'therapy': therapies,
                         'lab': lab.values.tolist()})
lab_test['lab'] = lab_test.lab.apply(lambda x: dict([i for i in x if i[0] is not np.nan]))
lab_test

In [ ]:
lab_test_1 = lab_test.merge(data, on=['study_id', 'ENCOUNTER_DT_RAN'])
lab_test_1[['study_id', 'ENCOUNTER_DT_RAN', 'therapy', 'current_therapy_x']]

In [ ]:
health_status = ehr.dropna(subset=['LAB_TEST'], inplace=False).groupby(['LAB_TEST']).count()[['study_id']].index.get_level_values(0).tolist()
health_status

In [ ]:
health_status = ehr.dropna(subset=['LAB_TEST'], inplace=False).groupby(['LAB_TEST']).count()[['study_id']].index.get_level_values(0).tolist()

for lab in health_status:
    lab_test[lab] = lab_test.apply(lambda _: _.lab[lab] if lab in _.lab.keys() else None, axis=1)

# for thrpy in diabetes_therapy_list:
#     lab_test[thrpy] = lab_test.apply(lambda _: 1.0 if thrpy in _.therapy else 0.0, axis=1)

# for basic_test, idx in {'BP_SYSTOLIC': 2, 'BP_DIASTOLIC':3 ,'BMI':4, 'WEIGHT':5, 'RX_CATEGORY': 6, 'THERA_CLASS': 7}.items():
#     lab_test[basic_test] = lab_test.apply(lambda _: _.status_y[idx], axis=1)

lab_test

In [ ]:
lab_test.loc[lab_test.study_id =='19175',].groupby(['study_id'])

In [ ]:
lab_test.drop(['therapy', 'lab'], axis=1).to_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_full_No_CVD_EHR_data.csv', index=False)

In [ ]:
lab_test = pd.read_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_full_No_CVD_EHR_data.csv', delimiter=',')

In [ ]:
lab_test = lab_test.sort_values(by=['study_id', 'ENCOUNTER_DT_RAN'])#.merge(treatment_mapping,left_on=['therapy'], right_on=['PHARM_CLASS'])

In [ ]:
#therapy_df = ehr[['ENCOUNTER_DT_RAN', 'study_id','PHARM_CLASS','THERA_CLASS','RX_CATEGORY']].drop_duplicates()
#ehr_diabetes = ehr.loc[ehr['RX_CATEGORY'] == 'Diabetes RX',]
therapy_diabetes_df = ehr.groupby(['ENCOUNTER_DT_RAN', 'study_id'])['PHARM_CLASS'].apply(list) #'THERA_CLASS','RX_CATEGORY'
#therapy_df.study_id = therapy_df.study_id.astype('int64')
# therapy_df = pd.DataFrame({'ENCOUNTER_DT_RAN': lab.index.get_level_values(0).tolist(),
#                          'study_id': lab.index.get_level_values(1).tolist(),
#                          'therapy': lab.index.get_level_values(2).tolist(),
#                          'RX_CATEGORY': lab.index.get_level_values(3).tolist(),
#                           'THERA_CLASS': lab.index.get_level_values(4).tolist()})
therapy_gdf = pd.DataFrame({'ENCOUNTER_DT_RAN': therapy_diabetes_df.index.get_level_values(0).tolist(),
                         'study_id': therapy_diabetes_df.index.get_level_values(1).tolist(),
                         'PHARM_CLASS': therapy_diabetes_df.values.tolist()})

therapy_gdf_PHARM_CLASS = []
for i in therapy_gdf.PHARM_CLASS:
    if i[0] is not np.nan:
        therapy_gdf_PHARM_CLASS.append(i)
    else:
        therapy_gdf_PHARM_CLASS.append([])
        
therapy_gdf['PHARM_CLASS'] = therapy_gdf_PHARM_CLASS
therapy_gdf


In [ ]:
import time
cnt= 0
timestamp = time.time()
treatment_hist_log = []
treatment_dates_patients = []
# treatment_dates = []
for patient, d in therapy_gdf.groupby(['study_id']):
    if cnt % 100 == 0: 
        print(cnt)
        print(time.time() - timestamp)
        timestamp = time.time()
        
    cnt += 1
    last_treatment = d.iloc[0]['PHARM_CLASS']
    total_num_visits = d.shape[0]
    for visit_idx in range(d.shape[0]):
        cur_treatment = d.iloc[visit_idx]['PHARM_CLASS']
        if len(cur_treatment) == 0:
            cur_treatment = last_treatment
        else:
            last_treatment = cur_treatment
        
        treatment_hist_log.append(cur_treatment) #
        treatment_dates_patients.append(d.iloc[visit_idx][['study_id','ENCOUNTER_DT_RAN']].tolist())
    
    
#         treatment_patients = d.iloc[visit_idx][['study_id']].tolist()

treatment_data = pd.DataFrame(treatment_dates_patients, columns=['study_id','ENCOUNTER_DT_RAN'])
# treatment_data['ENCOUNTER_DT_RAN'] = treatment_dates
treatment_data['PHARM_CLASS'] = treatment_hist_log
# treatment_data = pd.DataFrame(treatment_hist_log, columns=['THERA_CLASS'])


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(treatment_data['PHARM_CLASS'].tolist())
labels = [cl.replace('\n','').replace(' ','_') for cl in mlb.classes_]
for lb in range(len(labels)):
    treatment_data[labels[lb]] = mlb.transform(treatment_data['PHARM_CLASS'].tolist())[:,lb]

In [ ]:
treatment_data.drop('PHARM_CLASS',axis=1).to_csv('cleaned_EHR_treatment_param.csv')